In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

### The first step in working with any dataset is loading the data in and noting what information is included in the dataset. This is an important step in eventually working with any data, and knowing what kinds of features we have to work with. In this notebook, we'll explore the data and observe the patterns in the data.

1- Read data
2- Study each attribute and it's characteristics:
    -Name
    -Type
    -Missing values
    -Noisiness and type (stochastic, outliers, rounding errors, etc)
    -Type of distribution
3- Identify target attributes
4- Visualize the data
5- Study correlation between attributes
6- Identify promissing transformations

In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# magic word for producing visualizations in notebook
%matplotlib inline

In [7]:
azdias = pd.read_csv('azdias.csv')
customers = pd.read_csv('customers.csv')

C:\Users\Ahmad Abd Elhameed\Anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# number of rows and columns in dataset
print("AZDIAS Shape: {}".format(azdias.shape))
print("CUSTOMER Shape: {}".format(customers.shape))

AZDIAS Shape: (891221, 367)
CUSTOMER Shape: (191652, 370)


In [9]:
azdias.head()

,Unnamed: 0,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [10]:
customers.head()

,Unnamed: 0,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [21]:
values = pd.read_excel("DIAS Attributes - Values 2017.xlsx",engine='openpyxl',header = 0)

In [22]:
values.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Attribute,Description,Value,Meaning
1,NaN,AGER_TYP,best-ager typology,-1,unknown
2,NaN,NaN,NaN,0,no classification possible
3,NaN,NaN,NaN,1,passive elderly
4,NaN,NaN,NaN,2,cultural elderly


In [25]:
values.drop('Unnamed: 0',axis = 1,inplace = True)

In [27]:
values.columns = ["Attribute","Description","Value","Meaning"]

In [28]:
# Drop first row 
# by selecting all rows from first row onwards
values = values.iloc[1: , :]

In [30]:
values.head(10)

,Attribute,Description,Value,Meaning
1,AGER_TYP,best-ager typology,-1,unknown
2,NaN,NaN,0,no classification possible
3,NaN,NaN,1,passive elderly
4,NaN,NaN,2,cultural elderly
5,NaN,NaN,3,experience-driven elderly
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
7,NaN,NaN,1,< 30 years
8,NaN,NaN,2,30 - 45 years
9,NaN,NaN,3,46 - 60 years
10,NaN,NaN,4,> 60 years


In [31]:
info = pd.read_excel("DIAS Information Levels - Attributes 2017.xlsx",engine='openpyxl',header = 0)

In [33]:
info.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Information level,Attribute,Description,Additional notes
1,NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
2,NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
3,NaN,NaN,ANREDE_KZ,gender,NaN
4,NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."


In [34]:
info.drop('Unnamed: 0',axis = 1,inplace = True)
info.columns = ["Information level","Attribute","Description","Additional notes"]
# Drop first row 
# by selecting all rows from first row onwards
info = info.iloc[1: , :]
info.head()

,Information level,Attribute,Description,Additional notes
1,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
2,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
3,NaN,ANREDE_KZ,gender,NaN
4,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
5,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [35]:
values.ffill(inplace=True) # Fill the NaN rows with description or attribute
values['Value'] = values['Value'].astype('str') # Changing datatype from object to str

values.head(25)

C:\Users\Ahmad Abd Elhameed\Anaconda3\envs\nlp\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
C:\Users\Ahmad Abd Elhameed\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Attribute,Description,Value,Meaning
1,AGER_TYP,best-ager typology,-1,unknown
2,AGER_TYP,best-ager typology,0,no classification possible
3,AGER_TYP,best-ager typology,1,passive elderly
4,AGER_TYP,best-ager typology,2,cultural elderly
5,AGER_TYP,best-ager typology,3,experience-driven elderly
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
7,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years
8,ALTERSKATEGORIE_GROB,age classification through prename analysis,2,30 - 45 years
9,ALTERSKATEGORIE_GROB,age classification through prename analysis,3,46 - 60 years
10,ALTERSKATEGORIE_GROB,age classification through prename analysis,4,> 60 years


In [38]:
meta_attributes = values['Attribute'].unique()

print("Total Number of unique attributes given to us in meta data: ",len(meta_attributes))
print("Total Number of unique attributes AZIDAS data: ",azdias.columns.size)
print("Total Number of unique attributes CUSTOMERS data: ",customers.columns.size)


Total Number of unique attributes given to us in meta data:  314
Total Number of unique attributes AZIDAS data:  367
Total Number of unique attributes CUSTOMERS data:  370


The number of attributes given is not same in the three datasets. This means some of the attributes were not important enough to be included in the meta data.

In [41]:
# Finding common attributes bettween AZDIAS and Customers
common_azdias_customers = list(set(customers.columns).intersection(azdias.columns))
diffr_azdias_customers = list(set(customers.columns).difference(azdias.columns))

In [42]:
print("Number of attributes common between Customer and AZDIAS: {}".format(len(common_azdias_customers)))
print("Number of attributes exclusive to Customer: {}".format(len(diffr_azdias_customers)))
print("Attributes exclusive to CUSTOMERS: {}".format(diffr_azdias_customers))

Number of attributes common between Customer and AZDIAS: 367
Number of attributes exclusive to Customer: 3
Attributes exclusive to CUSTOMERS: ['PRODUCT_GROUP', 'ONLINE_PURCHASE', 'CUSTOMER_GROUP']


**ONLINE_PURCHASE**, **PRODUCT_GROUP** and **CUSTOMER_GROUP** are the attributes which are not in AZDIAS dataframe, but both datasets share rest of the features.

In [43]:
attr_common_all = set(common_azdias_customers).intersection(meta_attributes)
attr_common_customers = set(customers.columns).intersection(meta_attributes)
attr_cust_diffr_meta = set(customers.columns).difference(meta_attributes)
attr_meta_diffr_cust = set(meta_attributes).difference(customers.columns)

print('Len of total common attributes in all 3 tables',len(attr_common_all))
print('Len of attributes common in customers and meta data',len(attr_common_customers))
print('Len of attributes in Customers but not in meta',len(attr_cust_diffr_meta))
print('Len of attributes in meta data but not in customers',len(attr_meta_diffr_cust))


Len of total common attributes in all 3 tables 272
Len of attributes common in customers and meta data 272
Len of attributes in Customers but not in meta 98
Len of attributes in meta data but not in customers 42


There are 98 attributes in customers data that are not present in meta-data. Now, let's see what attributes are these.

In [49]:
print("Attributes exclusive to customers data","\n","\n".join(list(attr_cust_diffr_meta)))
print("******************************************************")
print("******************************************************")
print("******************************************************")
print("******************************************************")
print("Attributes exclusive to meta data","\n","\n".join(list(attr_meta_diffr_cust)))

Attributes exclusive to customers data 
 CAMEO_INTL_2015
D19_KONSUMTYP_MAX
D19_BILDUNG
RT_KEIN_ANREIZ
D19_DIGIT_SERV
UMFELD_ALT
D19_BEKLEIDUNG_REST
D19_TELKO_REST
KBA13_ANTG4
ANZ_STATISTISCHE_HAUSHALTE
GEMEINDETYP
D19_FREIZEIT
CUSTOMER_GROUP
D19_VOLLSORTIMENT
D19_ENERGIE
KBA13_ANTG3
KBA13_GBZ
D19_BANKEN_DIREKT
HH_DELTA_FLAG
D19_TELKO_MOBILE
D19_DROGERIEARTIKEL
CJT_TYP_2
D19_TELKO_ONLINE_QUOTE_12
EINGEZOGENAM_HH_JAHR
KBA13_CCM_1401_2500
CJT_TYP_6
EINGEFUEGT_AM
D19_BUCH_CD
MOBI_RASTER
D19_BIO_OEKO
D19_SOZIALES
D19_BEKLEIDUNG_GEH
VERDICHTUNGSRAUM
D19_RATGEBER
VHA
D19_VERSI_ONLINE_QUOTE_12
D19_SONSTIGE
D19_BANKEN_GROSS
KOMBIALTER
UMFELD_JUNG
ALTER_KIND1
D19_HANDWERK
D19_LOTTO
D19_TIERARTIKEL
FIRMENDICHTE
ALTER_KIND4
STRUKTURTYP
D19_HAUS_DEKO
AKT_DAT_KL
D19_SAMMELARTIKEL
D19_VERSI_OFFLINE_DATUM
VHN
D19_WEIN_FEINKOST
CJT_TYP_4
D19_VERSAND_REST
D19_LEBENSMITTEL
PRODUCT_GROUP
KBA13_HHZ
D19_VERSICHERUNGEN
LNR
VK_ZG11
D19_BANKEN_LOKAL
CJT_KATALOGNUTZER
ALTER_KIND3
D19_VERSI_ONLINE_DATUM
KBA13_AN

CAMEO_DEUINTL_2015 in attributes dataset is names as CAMEO_INTL_2015 in customers dataset.

## Missing data
### Identifying missing data

In this section, we'll identify the missing data which is represented using other values given in meta data other irregularities.

In [ ]:
attributes = values.loc[values['Attribute'].isin(attr_common_all)]